In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Results and Outlook

The goal of work was to create an engine that could keep up with good player with an ELO around 1500. Although it is hard to measure exactly the ELO of our engine, the test results have shown that our prototype v2 engine at a depth of 4 is perhaps even slightly stronger than this. Furthermore, it is also able to play quite fast at that level in contrast the prototype v1 engine.

In [ ]:
from converted_notebooks.s17_prototype_v2 import s17_PrototypeV2Engine_results, s17_PrototypeV2Engine_see_results, stockfish_results
import pandas as pd
import IPython.display

results = s17_PrototypeV2Engine_results + s17_PrototypeV2Engine_see_results
results_frame = pd.DataFrame(results)

concatenated_result_frames = pd.concat([
    result[1] for result in stockfish_results
])

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    IPython.display.display(results_frame.groupby(['engine', 'depth']).sum())
    IPython.display.display(concatenated_result_frames)

Nevertheless, there is still a lot of room for improvement. First of all there are various ways how the code itself can be optimized. The most important aspect is perhaps to use a more optimized chess library and perhaps in general another language such as C++. But there have also been some design decision in order to make the code more understandable or be able to introduce new things in a more didactic approach that are not ideal. One of this is the implementation of the cache decorator. Storing values in the cache should rather be done in the `_value` function for multiple reasons. First we can save some code to determine the NodeType, because we can set the node type directly in the `_value` function. By default, it will always be an `ALL` node. If any move is able to increase alpha then we know that we have an `EXACT` node type. And if there is a beta cutoff then the node type must be a `CUT`. But more important than this is that we actually know the corresponding moves for the scores and can directly store the best move in the cache. With this, move ordering could be implemented much more efficiently.

There are also a lot more enhancements regarding the used algorithms possible. For instance there is the [MTD(f)](https://www.chessprogramming.org/MTD(f)) algorithm that can be used as an alternative to the Principal Variation Search. Or the move ordering could be enhanced. The chessprogramming wiki defines [several more ways](https://www.chessprogramming.org/Move_Ordering#Typical_move_ordering) what can be taken into account for move ordering. Very important as well is also the evaluation function. The one used here is already very efficient, but does not take into account many other heuristics that many chess players use to evaluate a move. For instance if rooks are positioned on open lines. Then there is also a category of changes that are trade offs and might improve the engines strength or might not. One of them is what moves should be considered in the quiescence search. Most engines do not only consider captures, but also checks. This can detect some case with zugzwang, but also increases the search space. 

Another very important aspect any chess engine that has not been considered in this work, is parallelism. The search could be done on multiple threads and speed up significantly. There are a lot of considerations with multi threading such as thread safety and when it's actually useful to calculate something in a different thread. The node types can help here again. For instance an `ALL` Node is safe to parallelize as all moves have to be analyzed anyway. On a `CUT` Node on the other hand that might not be useful as later moves would have been pruned anyway.

In conclusion, it would probably be possible to further enhance this engine and reach an ELO of 2000 without changing the language. If the engine was written in a more efficient language higher scores might be realistic as well.